In [2]:
import opendatasets as od
dataset_url = 'https://www.kaggle.com/daveianhickey/2000-16-traffic-flow-england-scotland-wales'
od.download(dataset_url)

100%|██████████| 132M/132M [00:36<00:00, 3.79MB/s] 


In [6]:
import pandas as pd


In [7]:
df = pd.read_csv(r'2000-16-traffic-flow-england-scotland-wales/ukTrafficAADF.csv')

In [16]:
readingCrash = df.loc[df['LocalAuthority'] == "Reading"]


In [17]:
readingCrash['Road'].unique()

array(['A4', 'A327', 'A329', 'A4155', 'M4', 'A33', 'A4074'], dtype=object)

In [21]:
df.head()

,AADFYear,CP,Estimation_method,Estimation_method_detailed,Region,LocalAuthority,Road,RoadCategory,Easting,Northing,...,V2AxleRigidHGV,V3AxleRigidHGV,V4or5AxleRigidHGV,V3or4AxleArticHGV,V5AxleArticHGV,V6orMoreAxleArticHGV,AllHGVs,AllMotorVehicles,Lat,Lon
0,2000,6007,Counted,Manual count,Yorkshire and the Humber,Rotherham,M1,TM,446000,389300,...,5351,656,711,1588,4715,2149,15170,111146,53.398531,-1.309655
1,2000,6009,Counted,Manual count,Yorkshire and the Humber,Leeds,M621,TM,432150,429500,...,1293,272,287,628,1494,921,4895,65939,53.760869,-1.513814
2,2000,6035,Counted,Manual count,Yorkshire and the Humber,Doncaster,M18,TM,466400,407900,...,1952,257,239,745,3695,2000,8888,40229,53.563523,-0.998970
3,2000,6054,Counted,Manual count,Yorkshire and the Humber,Calderdale,M62,TM,404000,416600,...,3121,508,354,1786,5902,3438,15109,85759,53.645893,-1.940967
4,2000,6055,Counted,Manual count,Yorkshire and the Humber,Leeds,M62,TM,426000,426200,...,4030,624,423,1947,6669,3743,17436,98487,53.731551,-1.607374
